In [1]:
import os

if "README.md" not in os.listdir():
    os.chdir("../")

In [2]:
from typing import Optional, List

from llama_cpp import Llama
from tqdm.autonotebook import tqdm
from pydantic import BaseModel, Field
from langchain import PromptTemplate
from huggingface_hub import hf_hub_download
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.output_parsers import YamlOutputParser


model_path = hf_hub_download(
    repo_id="VlSav/Vikhr-Nemo-12B-Instruct-R-21-09-24-Q4_K_M-GGUF",
    filename="vikhr-nemo-12b-instruct-r-21-09-24-q4_k_m.gguf",
    local_dir="./models",
)

/tmp/ipykernel_672073/2151348226.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


При выполнении задания не будут использованы цепи langchain, потому что langchain_community.llms.LlamaCpp криво обрабатывает промпты для VlSav/Vikhr-Nemo-12B-Instruct, что снижает качество генерации. Кроме того, ChatPromptTemplate не содержит роли documents, на которой специально обучался [Vikhr-Nemo-12B-Instruct](https://huggingface.co/Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24)

In [3]:
llm = Llama(
    model_path=model_path, max_tokens=2048, n_gpu_layers=-1, n_ctx=2048, verbose=False
)

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (1024000) -- the full capacity of the model will not be utilized


In [4]:
def create_chat(documents, user_message):
    return [
        {
            "role": "system",
            "content": "Ты специалист по анализу договоров. Твоя задача проанализировать договор по инструкции пользователя.",
        },
        {"role": "documents", "content": documents},
        {"role": "user", "content": user_message},
    ]


def generate(documents, user_message):
    return llm.create_chat_completion(
        create_chat(documents, user_message), temperature=0.2, seed=10
    )["choices"][0]["message"]["content"]

In [5]:
generate("", "Лучше биба или боба?")

'Это риторический вопрос, который не имеет однозначного ответа, так как предпочтения в еде субъективны и зависят от личных вкусов каждого человека. Оба продукта имеют свои преимущества и могут быть полезны в различных кулинарных контекстах.'

In [6]:
with open("./data/raw/dkp.txt", "r", encoding="utf-8") as f:
    doc = f.read()

# Используем сплиттер для разделения по нумерованным пунктам
splitter = RecursiveCharacterTextSplitter(
    separators=[r"\d+\.\s"],
    chunk_size=500,
    chunk_overlap=50,
    keep_separator=True,
    is_separator_regex=True,
)

chunks = splitter.split_text(doc)
len(chunks)

17

Задача будет решаться следующим образом:
- сначало определяем какая информация содержится в части текста:
    - стороны договора
    - о юридическом лице, долю которого передают по договору
    - договорная стоимость
    - номинальная стоимость
    - показатель доли

- в дальнейшем для каждого типа информации будет свой промпт


Использование отдельных промптов для разных видов информации упростит отладку промптов и приведет к меньшему количеству ошибок при работе llm.


Результат будет выводится в формате yaml, потому что при генерации yaml разметки, ошибок меньше, чем при генерации json.

In [7]:
class ClassifierScheme(BaseModel):
    answer: Optional[List[str]] = Field(default="")


prompt_classifier = """Тебе передана небольшая часть договора купли продажи:

 Укажи какая тема содержится в рассматриваемой части:
- Информация о сторонах договора (продавце или покупателе): 
    - ФИО стороны (если это физическое лицо), ее название (если это юридическое лицо)
    - ИНН
    - ОГРН
    - адрес.
  Такую информацию обозначай как 'parties'. Если указано только ФИО или название юридического лица, то не указывай эту тему в ответе.
- Предмет юридического лица. Информация о юридическом лице, долю которого передают по договору.
  Чтобы часть договора была отнесена к этой теме, она обязательно должна содержать ВСЮ следующую информацию о юридическом лице:
  - наименование
  - ИНН
  - ОГРН
  - КПП
  - адрес
  Такую информацию обозначай как 'subject'. Если из всех перечисленных данных указано только наименование юридического лица, то не указывай эту тему в ответе.
- Договорная стоимость - это реальная стоимость отчуждаемой доли, за которую стороны договорились произвести отчуждение.
  Такую информацию обозначай как 'contract_cost'.
- Номинальная стоимость - реальная стоимость отчуждаемой доли, за которую стороны договорились произвести отчуждение. Должен быть указан явно указана стоимость доли!
  Такую информацию обозначай как 'nom_cost'.
- Показатель доли. Это размер отчуждаемой доли. Должен быть указан явно указан размер отчуждаемой доли!
  Такую информацию обозначай как 'fraction'.

Учти, что часть договора может содержать несколько тем. Тогда надо будет указать на наличие их всех. Если ничего ни одна из указанных тем не раскрыта, то просто отвечай пустым списком, таким образом - answer: [].
Перед ответом напиши своим рассуждения.

Ниже пример ответа:
```yaml
    answer:
      - parties
      - fraction
```

{format_instructions}
"""

parser_classifier = YamlOutputParser(pydantic_object=ClassifierScheme)

prompt_classifier = (
    PromptTemplate(
        template=prompt_classifier,
        partial_variables={
            "format_instructions": parser_classifier.get_format_instructions(),
        },
    )
    .format_prompt()
    .text
)

text = generate(chunks[1], prompt_classifier)
print(text)
parser_classifier.parse(text)

В представленной части договора содержится информация о юридическом лице, долю в уставном капитале которого планируется передать. Следовательно, эта часть относится к теме 'subject'.

```yaml
answer:
  - subject
```


ClassifierScheme(answer=['subject'])

In [8]:
chunks[2]

'2. \xa0\xa0\xa0Размер отчуждаемой доли в уставном капитале ОБЩЕСТВА С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "КОМИНТЕРН" составляет 100% (сто процентов), номинальной стоимостью 10000 (десять тысяч) рублей 00 копеек. Отчуждаемая доля полностью оплачена Продавцом на момент подписания настоящего договора, что подтверждается: Справкой № б/н, выданной ОБЩЕСТВОМ С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "КОМИНТЕРН" 01 декабря 2020 года.'

In [9]:
class Address(BaseModel):
    country: Optional[str] = Field(default="")
    state: Optional[str] = Field(default="")
    locality: Optional[str] = Field(default="")
    street: Optional[str] = Field(default="")
    house: Optional[str] = Field(default="")


class Party(BaseModel):
    name: Optional[str] = Field(default="")
    passport: Optional[str] = Field(default="")
    inn: Optional[str] = Field(default="")
    ogrn: Optional[str] = Field(default="")
    address: Optional[Address] = Field(default_factory=Address)


class PartiesScheme(BaseModel):
    seller: Optional[Party] = Field(default_factory=Party)
    buyer: Optional[Party] = Field(default_factory=Party)


prompt_parties = """
Тебе передана небольшая часть договора купли продажи. Твоя задача определить в этой части информацию о сторонах договора (продавце и покупателе).
К этой информации относится:
  - <сторона сделки значение может быть только seller (Продавец) или buyer (Покупатель):
    - name // ФИО стороны (если это физическое лицо), ее название (если это юридическое лицо). Пиши в именительном падеже.
    - passport // Серия и номер паспорта. Написать необходимо слитно единой последовательностью. Например: '1111111111'. Запись должна быть в кавычках.
    - inn // ИНН, это индивидуальный номер налогоплательщика, состоит только из цифр. Для физических лиц состоит из 12 цифр, для юридических из 10 цифр.
    - ogrn // ОГРН, это основной государственный регистрационный номер, состоит только из цифр. Состоит из 13 цифр.
    - address: // указывай адрес места жительства для физического лица, адрес регистрации юридического лица.
      - country // страна места. Всегда пиши полное наименование. Например, не "Россия", а "Российская Федерация"
      - state // область, край, субъект.
      - locality // населенный пункт. Пиши только название населенного пункта, без его типа. Например, не "г. Москва", а "Москва".
      - street // улица
      - house // дом

Если информации нет, то просто пиши пустую строку "".

Учитывай, что Москва - это и субъект и населенный пункт, то есть если locality "Москва", то и state "Москва".

Сгенерируй yaml документ с описанием продавца и покупателя. Перед генерацией можешь написать свои рассуждения если необходимо.

{format_instructions}
"""
parties_parser = YamlOutputParser(pydantic_object=PartiesScheme)

prompt_parties = (
    PromptTemplate(
        template=prompt_parties,
        partial_variables={
            "format_instructions": parties_parser.get_format_instructions(),
        },
    )
    .format_prompt()
    .text
)

text = generate(chunks[0], prompt_parties)
print(text)
parties_parser.parse(text)

```yaml
seller:
  name: "Троцкий Лев Давидович"
  passport: "1234567890"
  address:
    country: "Российская Федерация"
    state: "Москва"
    locality: "Москва"
    street: ""
    house: ""

buyer:
  name: "Бухарин Николай Иванович"
  passport: "0987654321"
  address:
    country: "Российская Федерация"
    state: "Москва"
    locality: "Москва"
    street: ""
    house: ""
```


PartiesScheme(seller=Party(name='Троцкий Лев Давидович', passport='1234567890', inn='', ogrn='', address=Address(country='Российская Федерация', state='Москва', locality='Москва', street='', house='')), buyer=Party(name='Бухарин Николай Иванович', passport='0987654321', inn='', ogrn='', address=Address(country='Российская Федерация', state='Москва', locality='Москва', street='', house='')))

In [10]:
chunks[0]

'12 АА 3456789 ДОГОВОР КУПЛИ-ПРОДАЖИ ДОЛИ В УСТАВНОМ КАПИТАЛЕ ОБЩЕСТВА Город Москва Первое декабря две тысячи десятого года Мы, Троцкий Лев Давидович, 01 декабря 1950 года рождения, место рождения: гор. Москва, гражданство: Российской Федерации, пол: мужской, паспорт 12 34 567890, именуемый в дальнейшем «Продавец» с одной стороны, и Бухарин Николай Иванович, 01 апреля 1990 года рождения, место рождения: гор. Москва, гражданство: Российской Федерации, пол: мужской, паспорт 0987 654321, именуемый в дальнейшем «Покупатель», с другой стороны, заключили настоящий договор о нижеследующем: '

In [11]:
class SubjectScheme(BaseModel):
    name: Optional[str] = Field(default="")
    inn: Optional[str] = Field(default="")
    ogrn: Optional[str] = Field(default="")
    address: Optional[Address] = Field(default_factory=Address)


prompt_subject = """
Тебе передана небольшая часть договора купли продажи. Твоя задача определить в этой части информацию о юридическом лице, долю которого передают по договору.
К этой информации относится:
    - name // Наименование юридического лица. Пиши в именительном падеже.
    - inn // ИНН, это индивидуальный номер налогоплательщика, состоит только из цифр. Для юридических из 10 цифр.
    - ogrn // ОГРН, это основной государственный регистрационный номер, состоит только из цифр. Состоит из 13 цифр.
    - address: // адрес регистрации юридического лица
        - country // страна места. Всегда пиши полное наименование. Например, не "Россия", а "Российская Федерация".git/FETCH_HEAD
        - state // область, край, субъект.
        - locality // населенный пункт
        - street // улица
        - house // дом

Если информации нет, то просто пиши пустую строку "".

Учитывай, что Москва - это и субъект и населенный пункт, то есть если locality "Москва", то и state "Москва".

Сгенерируй yaml документ с ответом. Перед генерацией можешь написать свои рассуждения если необходимо.

{format_instructions}
"""

subject_parser = YamlOutputParser(pydantic_object=SubjectScheme)

prompt_subject = (
    PromptTemplate(
        template=prompt_subject,
        partial_variables={
            "format_instructions": subject_parser.get_format_instructions(),
        },
    )
    .format_prompt()
    .text
)
text = generate(chunks[1], prompt_subject)
print(text)
subject_parser.parse(text)

На основе предоставленной информации, можно определить следующее:

- Наименование юридического лица: ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "КОМИНТЕРН"
- ИНН: 1234567890
- ОГРН: 1234567890123
- Адрес регистрации: страна - Российская Федерация, субъект и населенный пункт - Москва, улица Ленина, дом № 1.

Теперь сгенерируем YAML документ с учетом указанных требований:

```yaml
name: "ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ КОМИНТЕРН"
inn: "1234567890"
ogrn: "1234567890123"
address:
  country: "Российская Федерация"
  state: "Москва"
  locality: "Москва"
  street: "Ленина"
  house: "1"
```


SubjectScheme(name='ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ КОМИНТЕРН', inn='1234567890', ogrn='1234567890123', address=Address(country='Российская Федерация', state='Москва', locality='Москва', street='Ленина', house='1'))

In [12]:
class ContractCostScheme(BaseModel):
    cost: Optional[str] = Field(default="")


prompt_contract_cost = """
Тебе передана небольшая часть договора купли продажи юридического лица. Твоя задача определить в этой части информацию о договорной стоимости.

Договорная стоимость – это реальная стоимость отчуждаемой доли, за которую стороны договорились произвести отчуждение. 
Необходимо, чтобы ответ был преобразован в следующий формат: X…X,YY. Где X - рубль, Y – копейка.
Результат заключай в кавычки, например "1000.10".

Если информации нет, то просто пиши пустую строку "".

Сгенерируй yaml документ с ответом. Перед генерацией можешь написать свои рассуждения если необходимо.

Пример ответа: 
cost: "100.00"

{format_instructions}
"""

contract_cost_parser = YamlOutputParser(pydantic_object=ContractCostScheme)

prompt_contract_cost = (
    PromptTemplate(
        template=prompt_contract_cost,
        partial_variables={
            "format_instructions": contract_cost_parser.get_format_instructions(),
        },
    )
    .format_prompt()
    .text
)
text = generate(chunks[4], prompt_contract_cost)
print(text)
contract_cost_parser.parse(text)

В предоставленной части договора указано, что договорная стоимость отчуждаемой доли составляет 50000 (пятьдесят тысяч) рублей 50 копеек. Следовательно, договорная стоимость в формате X…X,YY будет "50000.50".

```yaml
cost: "50000.50"
```


ContractCostScheme(cost='50000.50')

In [13]:
class NomCostScheme(BaseModel):
    cost: Optional[str] = Field(default="")


prompt_nom_cost = """
Тебе передана небольшая часть договора купли продажи юридического лица. Твоя задача определить в этой части информацию о номинальной стоимости.

Номинальная стоимость – это номинальная стоимость отчуждаемой доли в уставном капитале. 
Необходимо, чтобы ответ был преобразован следующий формат: X…X,YY. Где X - рубль, Y – копейка.
Результат заключай в кавычки, например "1000.10".

Если информации нет, то просто пиши пустую строку "".

Сгенерируй yaml документ с ответом. Перед генерацией можешь написать свои рассуждения если необходимо.

Пример ответа: 
cost: "100.00"



{format_instructions}
"""


nom_cost_parser = YamlOutputParser(pydantic_object=NomCostScheme)

prompt_nom_cost = (
    PromptTemplate(
        template=prompt_nom_cost,
        partial_variables={
            "format_instructions": nom_cost_parser.get_format_instructions(),
        },
    )
    .format_prompt()
    .text
)
text = generate(chunks[2], prompt_nom_cost)
print(text)
nom_cost_parser.parse(text)

На основе предоставленной части договора, номинальная стоимость отчуждаемой доли в уставном капитале составляет 10000 рублей 00 копеек. Следовательно, в формате X…X,YY это будет "10000.00".

Теперь сгенерируем YAML документ с ответом:

```yaml
cost: "10000.00"
```


NomCostScheme(cost='10000.00')

In [16]:
class FractionScheme(BaseModel):
    fraction: Optional[str] = Field(default="")


prompt_fraction = """
Тебе передана небольшая часть договора купли продажи юридического лица. Твоя задача определить в этой части информацию о показатели доли.
    Показатель доли – это размер отчуждаемой доли / части доли в уставном капитале.
    Необходимо, чтобы ответ был преобразован в следующий формат: указывается значение без знаков и пробелов, дробная часть отделяется запятой, округление производится до 6 знаков после запятой.
    Максимальный размер доли это 1, минимальный 0. Результат заключай в кавычки, например "1.000000".


Если информации нет, то просто пиши пустую строку "".

Сгенерируй yaml документ с ответом. Перед генерацией можешь написать свои рассуждения если необходимо.

{format_instructions}
"""


fraction_parser = YamlOutputParser(pydantic_object=FractionScheme)

prompt_fraction = (
    PromptTemplate(
        template=prompt_fraction,
        partial_variables={
            "format_instructions": fraction_parser.get_format_instructions(),
        },
    )
    .format_prompt()
    .text
)
text = generate(chunks[2], prompt_fraction)
print(text)
fraction_parser.parse(text)

Для начала определим размер отчуждаемой доли в уставном капитале компании. Согласно предоставленной информации, размер доли составляет 100% от уставного капитала. Это означает, что вся компания продается целиком.

Теперь вычислим показатель доли, который равен размеру отчуждаемой доли, деленной на 100% (или 1 в десятичном выражении), так как вся доля продается, показатель будет равен 1.

```yaml
fraction: "1.000000"
```


FractionScheme(fraction='1.000000')

In [17]:
prom = {
    "classifier": (prompt_classifier, parser_classifier),
    "parties": (prompt_parties, parties_parser),
    "subject": (prompt_subject, subject_parser),
    "contract_cost": (prompt_contract_cost, contract_cost_parser),
    "nom_cost": (prompt_nom_cost, nom_cost_parser),
    "fraction": (prompt_fraction, fraction_parser),
}

In [18]:
res = []

for chunk in tqdm(chunks):
    (prompt, parser) = prom["classifier"]
    text = generate(chunk, prompt)
    tags = parser.parse(text).answer
    for tag in tags:
        (prompt, parser) = prom[tag]
        text = generate(chunk, prompt)
        res.append(parser.parse(text))

  0%|          | 0/17 [00:00<?, ?it/s]

### Постобработка

Объеденим результаты в единный json

In [19]:
postprocessing_keys = {
    PartiesScheme: "parties",
    SubjectScheme: "subject",
    ContractCostScheme: "contract_cost",
    NomCostScheme: "nom_cost",
    FractionScheme: "fraction",
}
postprocessing_keys

{__main__.PartiesScheme: 'parties',
 __main__.SubjectScheme: 'subject',
 __main__.ContractCostScheme: 'contract_cost',
 __main__.NomCostScheme: 'nom_cost',
 __main__.FractionScheme: 'fraction'}

In [20]:
def recursive_update_dict(updatable_dict, update):
    """
    Функция для рекурсивного обновления словаря.
    """
    for key, value in update.items():
        if key not in updatable_dict:
            updatable_dict[key] = value
        elif not value:
            continue
        elif type(value) == dict:
            recursive_update_dict(value, updatable_dict[key])
        else:
            updatable_dict[key] = value

In [21]:
final_result = {}
for i in res:
    json = i.dict()
    key_final = postprocessing_keys[type(i)]

    if key_final not in final_result:
        final_result[key_final] = json
    else:
        recursive_update_dict(final_result[key_final], json)

/tmp/ipykernel_672073/314838402.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json = i.dict()


In [22]:
print("Общий результат парсинга:")
final_result

Общий результат парсинга:


{'parties': {'seller': {'name': 'Троцкий Лев Давидович',
   'passport': '1234567890',
   'inn': '',
   'ogrn': '',
   'address': {'country': 'Российская Федерация',
    'state': 'Москва',
    'locality': 'Москва',
    'street': '',
    'house': ''}},
  'buyer': {'name': 'Бухарин Николай Иванович',
   'passport': '0987654321',
   'inn': '',
   'ogrn': '',
   'address': {'country': 'Российская Федерация',
    'state': 'Москва',
    'locality': 'Москва',
    'street': '',
    'house': ''}}},
 'subject': {'name': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ КОМИНТЕРН',
  'inn': '1234567890',
  'ogrn': '1234567890123',
  'address': {'country': 'Российская Федерация',
   'state': 'Москва',
   'locality': 'Москва',
   'street': 'Ленина',
   'house': '1'}},
 'fraction': {'fraction': '1.000000'},
 'nom_cost': {'cost': '10000.00'},
 'contract_cost': {'cost': '50000.50'}}

Мной поверх ТЗ была добавлена еще задача парсинга паспортных данных (номера и серии) сторон сделки, потому что они чаще упоминаются в договорах, чем инн